### Part I: SORT raw images by channel

In [ ]:
import os
import shutil
import glob

os.chdir('/Users/Alexander.Morano/Desktop/data/2025_01_24_NRXN3a/R') 

# Set your prefix
prefix = 'scan'  # Replace this with your current prefix

# Step 1: List all TIF files matching the pattern
files = glob.glob(f"{prefix}_Plate_R_*.TIF")


# Step 2: Extract the 'd0', 'd1', or 'd2' from the filenames
prefixes = set()
for file in files:
    prematch = file.split('.')[0]  # Extract the relevant part like 'd0', 'd1', 'd2'
    match = prematch[-2:]
  
    if match.startswith('d') and match[1] in '012':
        prefixes.add(match)

# Step 3: For each unique prefix, create a directory and move the matching files
for prefix_match in sorted(prefixes):
    
    # Create directory if it doesn't exist
    os.makedirs(prefix_match, exist_ok=True)
    
    # Move matching files into the respective directory
    for file in files:
        if prefix_match in file:
            shutil.move(file, os.path.join(prefix_match, os.path.basename(file)))

print("Files have been organized.")


Part II: use CELLPOSE to identify GFP positive cells

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models, io, utils
from cellpose.io import imread
import os

modelpath = "/Users/Alexander.Morano/Desktop/cellpose_training/10x training/models/10X_NewCyto"
filepath = "/Users/Alexander.Morano/Desktop/data/2025_01_24_NRXN3a/R/d0"

os.chdir(filepath)
files = os.listdir()
print(len(files))

model = models.CellposeModel(gpu = False, pretrained_model = modelpath)
channels = [[0,0]]

for i in files:
    image = io.imread(i)
    masks, flows, styles = model.eval(image, diameter = 35.0, channels = channels)
    io.save_rois(masks, i)

### Part III: Rename and reoganize resulting files.

In [ ]:
import os

fileLocation = "/Users/Alexander.Morano/Desktop/data/2025_01_24_NRXN3a/R/d2/"
fileList = os.listdir(fileLocation)
print(fileList)

for ii in fileList:
    newName = ii.replace('d2','d0_rois')
    if newName != ii:
        os.rename(fileLocation+ii,fileLocation+newName)
print(fileList)

In [ ]:
import glob
import shutil

source_files = "/Users/Alexander.Morano/Desktop/data/2025_01_24_NRXN3a/R/d0/*.zip"
target_folder = "/Users/Alexander.Morano/Desktop/data/2025_01_24_NRXN3a/R/d2/"

# retrieve file list
filelist = glob.glob(source_files)
for single_file in filelist:
     # move file with full paths as shutil.move() parameters
    shutil.move(single_file,target_folder) 

### Part IV: After Running FIJI macro on output of 2+3; sort by well.

In [ ]:
import os
import shutil
import glob

os.chdir("/Users/Alexander.Morano/Desktop/data/2025_01_24_NRXN3a/R/results") 

# Set your prefix
prefix = 'scan'  
# Replace this with your current prefix

# Step 1: List all files matching the pattern
files = glob.glob(f"{prefix}_Plate_R_p00_0_*")

# Step 2: Extract the well name from this title.
prefixes = set()
for file in files:
    prematch = file.split('_')[5]
    
    match = prematch[:3]
    print(match)
  
    if match[0] in ('ABCDEFGH') and match[2] in '0123456789':
        prefixes.add(match)

# Step 3: For each unique prefix, create a directory and move the matching files
for prefix_match in sorted(prefixes):
    
    # Create directory if it doesn't exist
    os.makedirs(prefix_match, exist_ok=True)
    
    # Move matching files into the respective directory
    for file in files:
        if prefix_match in file:
            shutil.move(file, os.path.join(prefix_match, os.path.basename(file)))

print("Files have been organized AGAIN.")


Part V: Combine CSVs and produce output CSVs and graphs

In [ ]:
import os 
import pandas as pd
import matplotlib.pyplot as plt

root_path = "/Users/Alexander.Morano/Desktop/data/2025_01_24_NRXN3a/R/results/"

for root, sub, files in os.walk(root_path):
    filenames = [os.path.join(root, filename) for filename in files 
                 if filename.endswith('.csv')]
    
    flist = []
    plt.clf()
    for filename in filenames:
        print(os.path.join(root, filename))
        df = pd.read_csv(filename)
        flist.append(df)
        df_out = pd.concat(flist)
        df_out.loc['mean of cells'] = df_out.mean()
        df_out.to_csv(os.path.join(root, 'combined.csv'))
        fig = df_out['Mean'].hist(bins=[0, 100, 500, 1000, 2000, 3000, 4000])
        plt.savefig(os.path.join(root, 'fig'))
      

Part VI: add M images to the same subdirectories

In [ ]:
import os
import shutil
import glob

os.chdir("/Users/Alexander.Morano/Desktop/data/2025_01_24_NRXN3a/R/Results") 

# Set your prefix
prefix = 'scan'  # Replace this with your current prefix

# Step 1: List all files matching the pattern
files = glob.glob(f"{prefix}_Plate_M_p00_0_*")

# Step 2: Extract the well name from this title.
prefixes = set()
for file in files:
    prematch = file.split('_')[5]
    
    match = prematch[:3]
    print(match)
  
    if match[0] in ('ABCDEFGH') and match[2] in '0123456789':
        prefixes.add(match)

# Step 3: For each unique prefix, create a directory and move the matching files
for prefix_match in sorted(prefixes):
    
    # Create directory if it doesn't exist
    os.makedirs(prefix_match, exist_ok=True)
    
    # Move matching files into the respective directory
    for file in files:
        if prefix_match in file:
            shutil.move(file, os.path.join(prefix_match, os.path.basename(file)))

print("Files have been organized AGAIN.")

Part VII: Finally, you can generate a csv telling you which wells are positive and which are negative!

In [ ]:
import os 
import pandas as pd
import matplotlib.pyplot as plt

root_path = '/Users/Alexander.Morano/Desktop/data/2025_01_24_NRXN2a/R/Results'

import csv

#as long as we set up plate maps with 0.1 1 and 10 and each column corresponds to an antibody with a certain number of rows corresponding to an antigen, this will work 
#you just have to save 'ablist' as a list of abs on the plate as a csv
#then manually change the antigens because there are usually only a few.
#This part of the code is setting up other additions to the overall output file.

antibodylist = []
with open('/Users/Alexander.Morano/Desktop/data/2025_01_24_NRXN2a/R/ablist.csv') as inputfile:
    for row in csv.reader(inputfile):
        antibodylist.append(row[0])
        
antibodylist[0] = antibodylist[0][1:]

#Now, we're going to start combining data from each of the subfolders into one large data frame containing data from each well.

root_path = '/Users/Alexander.Morano/Desktop/data/2025_01_24_NRXN2a/R/Results'

overall_quant = []

for root, sub, files in os.walk(root_path):

   filenames = [os.path.join(root, filename) for filename in files 
                 if filename.startswith('combined')]
   
   for filename in filenames:
        print(os.path.join(root, filename))
        x = os.path.join(root, filename)
        y = x.split('/')[-2]
        df = pd.read_csv(filename)
        mean_combined = df['Mean'].mean()
       
        if mean_combined > 100: #you can change this value depending on the intensity of the far red channel
            result = 'Positive'
        else:
            result = 'Negative'
                
        overall_quant.append({'Subfolder': y, 'Data': mean_combined, 'Results': result})
        

oqdf = pd.DataFrame(overall_quant)

#now that we have this data frame, we can add the antibody name, concentration, and antigen  name to the same data frame. 
#below is a general way to create the "antibody" column assuming one antibody, with 3 concentrations,  2 species, and one control well, is being tested 
#per column of the 96 well plate. 

def antibody(subfolder): 
    #extract the first character of the well
    letter = subfolder[0]
    #convert the number part of the well to an integer
    number = int(subfolder[1:])
    
    if letter == 'A' and number == 1:
        return "V5_CTL"
    if letter == 'A' and number == 3:
        return "V5_CTL"
    elif letter == 'A' and number == 2:
        return "Isotype_CTL"
    elif letter == 'A' and number == 4:
        return "Isotype_CTL"
    
    elif letter >= 'B' and letter <= 'H' and number == 1:
        return antibodylist[0]
    elif letter >= 'B' and letter <= 'H' and number == 2:
       return antibodylist[1]
    elif letter >= 'B' and letter <= 'H' and number == 3:
      return antibodylist[2]
    elif letter >= 'B' and letter <= 'H' and number == 4:
      return antibodylist[3]
    elif letter >= 'B' and letter <= 'H' and number == 5:
      return antibodylist[4]
    elif letter >= 'B' and letter <= 'H' and number == 6:
      return antibodylist[5]
    elif letter >= 'B' and letter <= 'H' and number == 7:
      return antibodylist[6]
    elif letter >= 'B' and letter <= 'H' and number == 8:
      return antibodylist[7]
    elif letter >= 'B' and letter <= 'H' and number == 9:
      return antibodylist[8]
    elif letter >= 'B' and letter <= 'H' and number == 10:
      return antibodylist[9]
    elif letter >= 'B' and letter <= 'H' and number == 11:
      return antibodylist[10]
    elif letter >= 'B' and letter <= 'H' and number == 12:
      return antibodylist[11]

oqdf['Antibody'] = oqdf['Subfolder'].apply(antibody)

#now we can add antibody concentration to the data frame. 
#these will stay the same regardless of what the rest of the setup is. I am using integers instead of concentrations so we can do math later. 
#but each of the concentration values are in ug/mL 

def concentration(subfolder):
    letter = subfolder[0]
    number = int(subfolder[1:])
    
    if letter == 'A':
        return 1
    elif letter == 'D' or letter == 'G':
        return 10
    elif letter == 'C' or letter == 'F' or letter == 'H':
        return 1
    elif letter == 'B' or letter == 'E':
        return 0.1


oqdf['Concentration'] = oqdf['Subfolder'].apply(concentration)

#finally, we are going to add the antigen. These you have to change manually but it should not be very difficukt. 

def antigen(subfolder):
    letter = subfolder[0]
    number = int(subfolder[1:])
    
    if letter >= 'B' and letter <= 'D':
        return 'hNRXN2a'
    elif letter >= 'E' and letter <= 'G':
        return 'mNRXN2a'
    elif letter == 'H':
        return 'WY0041'
    
oqdf['Antigen'] = oqdf['Subfolder'].apply(antigen)

#now we can output the entire thing!

sorted_oqdf = oqdf.sort_values(by='Subfolder')
print(sorted_oqdf)
sorted_oqdf.to_csv(os.path.join(root_path,r'finalresultsalpha.csv'))


Part VIII: Generate graphs comparing mean fluorescence intensity and concentration! 

In [ ]:

import os
import sys
import glob
import shutil
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from datetime import date
import matplotlib.pyplot as plt
import argparse
import csv

root_path = '/Users/Alexander.Morano/Desktop/data/2025_01_24_NRXN2a/R/Results'

df = pd.read_csv("/Users/Alexander.Morano/Desktop/data/2025_01_24_NRXN2a/R/Results/finalresultsalpha.csv")

#this line is only necessary if you accidentally added the concentrations as strings instead of floats.
df.replace({'0.1_ug_mL': 0.1, '1_ug_mL': 1, '10_ug_mL': 10}, inplace=True)

unique_conditions = df['Antibody'].unique()

#This "filtered" version of the unique antibodies data frame is done to remove the controls (V5 and IGG)
filtered_unique_conditions = unique_conditions[1:]

for condition in filtered_unique_conditions:
    subset = df[df['Antibody'] == condition]
    
    human = subset[subset['Antigen'] == 'hNRXN2a']

    mouse = subset[subset['Antigen'] == 'mNRXN2a']
    control = subset[subset['Antigen'] == 'WY0041']
    fig, ax = plt.subplots(1) 
    handles, labels = ax.get_legend_handles_labels()
    ax.scatter(human['Concentration'], human['Data'], s=30, c='b', marker='o', label='human')
    ax.plot(human['Concentration'], human['Data'], c='b')
   
    ax.scatter(mouse['Concentration'], mouse['Data'], s=30, c='r', marker='o', label='mouse')
    ax.plot(mouse['Concentration'], mouse['Data'], c='r')
    ax.scatter(control['Concentration'], control['Data'], s=30, c='g', marker='o', label='Ctl')
    ax.set_title(condition)
    ax.set_xlabel('Concentration (uM)')
    ax.set_ylabel('MFI')
    ax.set_xscale('log')
    ax.legend(handles, labels)
    plt.legend() 
    plt.savefig(os.path.join(root_path,condition+'.png'), format = "png")
    plt.show()
    plt.close()

  

In [ ]:
Part IX: Organizing Files Again!

In [ ]:

import pandas as pd
import os
import shutil

root_path = '/Users/Alexander.Morano/Desktop/data/2025_01_24_NRXN2a/R/'

df = pd.read_csv("/Users/Alexander.Morano/Desktop/data/2025_01_24_NRXN3a/R/Results/finalresultsalpha.csv")

results_path = os.path.join(root_path, Results)
print(results_path)

for rownum, row in df.iterrows():
  # Get names
  subfolder_name = row['Subfolder']
  concentration = row['Concentration']
  antibody = row['Antibody']
  antigen = row['Antigen']
  
  ## Rename file in folder
  new_name = f"{antibody}_{concentration}_{antigen}"
  
  # Create the full path to the subfolder
  old_path = os.path.join(results_path, subfolder_name)
  new_path = os.path.join(results_path, new_name)
    
  # Step 4: Rename the folder
  if os.path.exists(old_path):
     os.rename(old_path, new_path)
     print(f"Renamed folder: {subfolder_name} -> {new_name}")
  else:
     print(f"Folder {subfolder_name} does not exist.")
     

#Next, let's sort into larger subfolders based on antibody name

#first: make a subfolder of "results" called "folders"
    
folders_path = os.path.join(root_path, folders) 

#now we have the folders "folders" and "results" in the "R" directory.

os.mkdir(folders_path) 
print("Directory '%s' created" %folders) 

unique_conditions = df['Antibody'].unique()

for condition in unique_conditions:
    
    dir_path = os.path.join(folders_path, condition)
    
    try:
        os.makedirs(dir_path, exist_ok=True)  # exist_ok=True prevents error if directory already exists
        print(f"Directory created: {dir_path}")
        
    except Exception as e:
        print(f"Error creating directory {condition}: {e}")
        


# Get the list of all folders in each directory
Results_Dirs = os.listdir(results_path)
Folders_Dirs = os.listdir(folders_path)

# Iterate over each folder in the first list
for folder in Results_Dirs:
    # Extract the first part of the folder name (before the first underscore)
    prefix = folder.split('_')[0]

    # Check if the prefix exists in the second list
    if prefix in Folders_Dirs:
        # Construct the full paths for the source and destination folders
        source_path = os.path.join(results_path, folder)
        destination_path = os.path.join(folders_path, prefix)

        # Move the folder into the corresponding folder in the second list
        try:
            # Ensure the destination folder exists
            if not os.path.exists(destination_path):
                print(f"Destination folder {destination_path} doesn't exist!")
            else:
                # Move the folder (it will merge into the second list folder)
                shutil.move(source_path, destination_path)
                print(f"Moved {folder} to {destination_path}")
                
        except Exception as e:
            print(f"Error moving {folder}: {e}")
    else:
        print(f"No matching folder found for {folder} in the second list")
        
#now let's move the graphs into there.
png_files = [f for f in os.listdir(results_path) if f.endswith('.png')]

# Iterate over each PNG file
for png_file in png_files:
    # Extract the prefix (name before the first underscore) from the PNG file name
    prefix = png_file.split('.png')[0]

    # Check if the prefix exists in the second list of folders
    if prefix in Folders_Dirs:
        # Construct the full paths for the source PNG file and the destination folder
        source_png_path = os.path.join(results_path, png_file)
        destination_folder_path = os.path.join(folders_path, prefix)

        # Move the PNG file into the corresponding folder in the second list
        try:
            # Ensure the destination folder exists
            if not os.path.exists(destination_folder_path):
                print(f"Destination folder {destination_folder_path} doesn't exist!")
            else:
                # Move the PNG file
                shutil.move(source_png_path, destination_folder_path)
                print(f"Moved {png_file} to {destination_folder_path}")
                
        except Exception as e:
            print(f"Error moving {png_file}: {e}")
    else:
        print(f"No matching folder found for {png_file} in the second list")